In [6]:
import os

from openai import OpenAI
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTASH_VECTOR_URL = os.environ.get('UPSTASH_VECTOR_URL')
UPSTASH_VECTOR_TOKEN = os.environ.get('UPSTASH_VECTOR_TOKEN')

In [3]:
import wikipedia
ny = wikipedia.page(title="new York City, New York")
# ny.content

In [7]:
from langchain_openai import OpenAIEmbeddings

# dim -> 1536
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# dim -> 3072
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [9]:
from langchain_community.vectorstores import UpstashVectorStore
store = UpstashVectorStore(
    embedding=embeddings,
    index=UPSTASH_VECTOR_URL,
    index_token=UPSTASH_VECTOR_TOKEN,
)

NameError: name 'embeddings' is not defined

In [10]:
wikipedia.search("Kathmandu")

['Kathmandu',
 'Kathmandu Valley',
 'Kathmandu (disambiguation)',
 'Nepal',
 'Kathmandu District',
 'Kathmandu–Terai Expressway',
 'Tribhuvan International Airport',
 'History of Kathmandu',
 'KMD Brands',
 'Kathmandu Durbar Square']

In [13]:
from langchain_core.documents import Document
documents = []
cities = ['New York City, New York', 'kathmandu']
for city in cities:
    wikipedia_page_result = wikipedia.page(title = city)
    doc = Document(
        page_content=wikipedia_page_result.content,
        metadata={
            "source": f"{wikipedia_page_result.url}",
            "title": wikipedia_page_result.title,
        },
    )
    documents.append(doc)
    # raw_documents.append(page.content)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City'}, page_content='New York, often called New York City or NYC, is the most populous city in the United States. Located at the southern tip of New York State on one of the world\'s largest natural harbors, the city comprises five boroughs, each coextensive with a respective county. New York is a global center of finance and commerce, culture, technology, entertainment and media, academics and scientific output, the arts and fashion, and, as home to the headquarters of the United Nations, international diplomacy.\nWith an estimated population in 2023 of 8,258,035 distributed over 300.46 square miles (778.2 km2), the city is the most densely populated major city in the United States. New York City has more than double the population of Los Angeles, the nation\'s second-most populous city. New York is the geographical and demographic center of both the Northeast megalopolis and the New York 

In [17]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter.from_tiktoken_encoder(model_name='gpt-4o', chunk_size=100, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)


NameError: name 'store' is not defined

In [ ]:
inserted_vectors = store.add_documents(docs)
result = store.similarity_search_with_score("The city with big buildings", k=5)
for doc, store in result:
    print(f"{doc}: {store}")